In [4]:
import pandas as pd
import sqlite3

In [5]:
conn = sqlite3.connect("E:/cbs.db")  # Creates a file called cbs.db

## Loading all transactions of 2025

In [ ]:
#csv_file = "./data/ACVWS_ALL_AC_ENTRIES_ACRJRNAL.csv"  # Update this path
#chunk_size = 500000  # You can adjust this based on memory

#for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    #chunk.to_sql("ACVWS_ALL_AC_ENTRIES_ACRJRNAL", conn, if_exists="append", index=False)

In [8]:
# Preview the first 5 rows
query = "SELECT * FROM ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2025 LIMIT 5"
df = pd.read_sql_query(query, conn)
df.head()

,AC_BRANCH,AC_CCY,AC_ENTRY_SR_NO,AC_NO,AML_EXCEPTION,AMOUNT_TAG,AUTH_ID,AUTH_STAT,BALANCE_UPD,BANK_CODE,...,RELATED_ACCOUNT,RELATED_CUSTOMER,RELATED_REFERENCE,STMT_DT,TRN_CODE,TRN_DT,TRN_REF_NO,TXN_INIT_DATE,USER_ID,VALUE_DT
0,20,TZS,646581397,LI146100,None,ARNGT_FEE_1_FACR,SYSTEM,A,D,None,...,020ECBL250030001,14245120.0,None,05/02/2025 00:00:00,251,05/02/2025 00:00:00,020ZTRF251220189,05/02/2025 00:00:00,SYSTEM,05/01/2025 00:00:00
1,12,TZS,646620204,IN110045,None,MAIN_INT_ACCR,SYSTEM,A,D,None,...,012ECBL242810503,14200922.0,None,05/02/2025 00:00:00,249,05/02/2025 00:00:00,012ZTRF2512204XY,05/02/2025 00:00:00,SYSTEM,05/01/2025 00:00:00
2,20,TZS,646581398,IN120130,None,ARNGT_FEE_1_FACR,SYSTEM,A,D,None,...,020ECBL250030001,14245120.0,None,05/02/2025 00:00:00,251,05/02/2025 00:00:00,020ZTRF25122018A,05/02/2025 00:00:00,SYSTEM,05/01/2025 00:00:00
3,20,TZS,646581399,AS185035,None,MAIN_INT_ACCR,SYSTEM,A,D,None,...,020ECBL250030501,14247656.0,None,05/02/2025 00:00:00,249,05/02/2025 00:00:00,020ZTRF25122018B,05/02/2025 00:00:00,SYSTEM,05/01/2025 00:00:00
4,11,TZS,646678794,LI145017,None,EXCISE_AMT,ESB001,A,D,None,...,None,14291908.0,None,05/02/2025 00:00:00,EXC,05/02/2025 00:00:00,011DAWM251220022,05/02/2025 00:00:00,ESB001,05/01/2025 00:00:00


## Importing multiple tables from a folder

In [ ]:
# import sqlite3
# import pandas as pd
# import os

# # Folder containing your CSV files
# csv_folder = './data_tables'
# db_name = 'cbs.db'

# # Connect to the SQLite database
# conn = sqlite3.connect(db_name)
# cursor = conn.cursor()

# # Loop through each CSV file in the folder
# for filename in os.listdir(csv_folder):
#     if filename.endswith('.csv'):
#         table_name = os.path.splitext(filename)[0]  # Use file name (without extension) as table name
#         file_path = os.path.join(csv_folder, filename)

#         # Check if the table already exists
#         cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?;", (table_name,))
#         exists = cursor.fetchone()

#         if exists:
#             print(f"Skipping: Table '{table_name}' already exists.")
#             continue

#         # Read CSV into DataFrame
#         df = pd.read_csv(file_path)

#         # Create table and insert data
#         df.to_sql(table_name, conn, if_exists='fail', index=False)
#         print(f"Created table: {table_name}")

# print("Table creation complete.")

# # Close connection
# conn.close()

## Creating table for All Customer lists registration

In [11]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text

# File and table setup
excel_file = './data_tables/CUSTOMER_LIST.csv'
table_name = 'customer_list'
db_file = 'my_database.db'

# Load Excel data into DataFrame
df = pd.read_csv(excel_file)

# Define column-specific padding rules
padding_rules = {
    'CUST_AC_NO': 12,
    'BRANCH_CODE': 3,
    'CUST_NO': 9
}

# Apply zero-padding and cast to string for each specified column
for col, width in padding_rules.items():
    if col in df.columns:
        df[col] = df[col].apply(lambda x: str(int(x)).zfill(width) if pd.notnull(x) else None)

# Create SQLite engine
engine = create_engine(f'sqlite:///{db_file}')

# Drop the old table if it exists
with engine.begin() as conn:
    conn.execute(text(f"DROP TABLE IF EXISTS {table_name}"))

# Write the modified DataFrame to SQLite
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

print(f"✅ Table '{table_name}' created with correct zero-padded TEXT fields.")

# Inspect the schema of the new table
inspector = inspect(engine)
columns = inspector.get_columns(table_name)

print(f"\n📋 Schema for table '{table_name}':")
for col in columns:
    print(f"{col['name']} - {col['type']}")


C:\Users\h.kemibara\AppData\Local\Temp\ipykernel_4300\666853815.py:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(excel_file)


✅ Table 'customer_list' created with correct zero-padded TEXT fields.

📋 Schema for table 'customer_list':
Unnamed: 0 - BIGINT
BRANCH_NAME - TEXT
BRANCH_CODE - TEXT
CUST_AC_NO - TEXT
CUST_NO - TEXT
CUSTOMER_TYPE - TEXT
CCY - TEXT
ACCOUNT_CLASS - TEXT
ACCOUNT_DESCRIPTION - TEXT
AC_STAT_DORMANT - TEXT
AC_OPEN_DATE - TEXT
CIF_CREATION_DATE - TEXT
LCY_CURR_BALANCE - FLOAT


In [12]:
df.head()

,Unnamed: 0,BRANCH_NAME,BRANCH_CODE,CUST_AC_NO,CUST_NO,CUSTOMER_TYPE,CCY,ACCOUNT_CLASS,ACCOUNT_DESCRIPTION,AC_STAT_DORMANT,AC_OPEN_DATE,CIF_CREATION_DATE,LCY_CURR_BALANCE
0,0,ROCK CITY,004,004000009868,004079947,I,TZS,PERSSA,PERSONAL SAVINGS ACCOUNT,Y,09/06/2010 00:00,16/11/2018 00:00,0.00
1,1,ROCK CITY,004,004000009879,004079952,C,TZS,SMBECA,SMALL BUSINESSES CURRENT ACCOUNT,N,09/06/2010 00:00,16/11/2018 00:00,9667.26
2,2,ROCK CITY,004,004000009968,004080124,I,TZS,DHAMSA,DHAMIRA SAVINGS ACCOUNT,Y,19/11/2014 00:00,16/11/2018 00:00,1788.90
3,3,ROCK CITY,004,004000009981,004080130,I,TZS,WATOSA,WATOTO SAVINGS ACCOUNT,N,18/06/2010 00:00,16/11/2018 00:00,0.00
4,4,ROCK CITY,004,004000010038,004080247,I,TZS,PERSSA,PERSONAL SAVINGS ACCOUNT,N,24/06/2010 00:00,16/11/2018 00:00,0.00


## Inspecting the whole database schema

In [6]:
## Inspecting the whole database schema
tables = inspector.get_table_names()

print(f"📂 Found {len(tables)} table(s) in the database:\n")

# Loop through each table and print its schema
for table in tables:
    print(f"📋 Schema for table '{table}':")
    columns = inspector.get_columns(table)
    for col in columns:
        print(f"  {col['name']} - {col['type']}")
    print("-" * 40)

📂 Found 10 table(s) in the database:

📋 Schema for table 'ACVWS_ALL_AC_ENTRIES_ACRJRNAL':
  AC_BRANCH - INTEGER
  AC_CCY - TEXT
  AC_ENTRY_SR_NO - INTEGER
  AC_NO - TEXT
  AML_EXCEPTION - REAL
  AMOUNT_TAG - TEXT
  AUTH_ID - TEXT
  AUTH_STAT - TEXT
  BALANCE_UPD - TEXT
  BANK_CODE - REAL
  BATCH_NO - REAL
  CATEGORY - INTEGER
  CURR_NO - INTEGER
  CUST_GL - TEXT
  DLY_HIST - TEXT
  DONT_SHOWIN_STMT - TEXT
  DRCR_IND - TEXT
  ENTRY_SEQ_NO - INTEGER
  EVENT - TEXT
  EVENT_SR_NO - INTEGER
  EXCH_RATE - REAL
  EXTERNAL_REF_NO - TEXT
  FCY_AMOUNT - REAL
  FINANCIAL_CYCLE - TEXT
  GLMIS_UPDATE_FLAG - REAL
  GLMIS_VAL_UPD_FLAG - TEXT
  IB - TEXT
  IC_BAL_INCLUSION - TEXT
  INSTRUMENT_CODE - TEXT
  LCY_AMOUNT - REAL
  MIS_FLAG - TEXT
  MIS_HEAD - TEXT
  MODULE - TEXT
  ORIG_PNL_GL - REAL
  PERIOD_CODE - TEXT
  PRINT_STAT - TEXT
  PRODUCT - TEXT
  PRODUCT_ACCRUAL - TEXT
  PRODUCT_PROCESSOR - TEXT
  RELATED_AC_ENTRY_SR_NO - INTEGER
  RELATED_ACCOUNT - TEXT
  RELATED_CUSTOMER - REAL
  RELATED_REF

## Loading transactions table with correct format for AC_BRANCH, RELATED_CUSTOMER

In [2]:
import sqlite3

# Path to your SQLite database file
db_path = "E:/cbs.db"

# Table name to delete
table_name = "ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2023"

# Connect and drop the table
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Drop the table (will delete it if it exists)
cursor.execute(f"DROP TABLE IF EXISTS {table_name};")

# Commit and close connection
conn.commit()
conn.close()

print(f"✅ Table '{table_name}' has been deleted (if it existed).")


✅ Table 'ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2023' has been deleted (if it existed).


In [ ]:
csv_file = ""  # Update this path
chunk_size = 500000  # You can adjust this based on memory

for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    chunk.to_sql("ACVWS_ALL_AC_ENTRIES_ACRJRNAL_2023", conn, if_exists="append", index=False)